### Importamos librerias

In [1]:
#!pip install beautifulsoup4 lxml numpy pandas
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

### Buscamos en Wikipedia

In [2]:
url = 'https://es.wikipedia.org/wiki/Sierra_Norte_(Madrid)'
response = requests.get(url)
response

<Response [200]>

### Pasamos la pagina a BeautifulSoup

In [3]:
pagina = bs(response.content, "html.parser")

### Separamos la comarca por subcomarcas y municipios

In [4]:
subcomarcas = {} # Diccionario para almacenar las subcomarcas
for text in pagina.select('div.thumbcaption span'):
    try:
        slicers = [text.text.strip('\xa0 ').index("("), text.text.strip('\xa0').index(':')] # Estos indices nos ayudarán a hacer slicing y eliminar caracteres no deseados
        subcomarca = (text.text.strip('\xa0 ')[:slicers[0]-1])
        subcomarcas[subcomarca] = (text.text.strip('\xa0 ')[slicers[1]+1:-1]).split('; ')
    except:
        pass
# Diccionario completo con {'Subcomarca' : [Municipios]}
#print(subcomarcas)

# Creamos el Dataframe de las subcomarcas
df_subcomarcas = pd.DataFrame.from_dict(subcomarcas, orient='index').T

# Lo guardamos como csv
df_subcomarcas.to_csv('./csv/Subcomarcas.csv', encoding='latin1')
df_subcomarcas.head()

,Sierra de La Cabrera,Valle del Jarama,Valle Alto del Lozoya,Valle Medio del Lozoya,Valle Bajo del Lozoya,Sierra del Rincón
0,Bustarviejo,El Vellón,Alameda del Valle,Braojos de la Sierra,Berzosa del Lozoya,Horcajuelo de la Sierra
1,Cabanillas de la Sierra,Patones,Canencia,Buitrago del Lozoya,Cervera de Buitrago,La Hiruela
2,La Cabrera,Redueña,Garganta de los Montes,Gascones,El Atazar,Montejo de la Sierra
3,Lozoyuela-Navas-Sieteiglesias,Torrelaguna,Gargantilla del Lozoya y Pinilla de Buitrago,Horcajo de la Sierra,El Berrueco,Prádena del Rincón
4,Navalafuente,Torremocha de Jarama,Lozoya,La Acebeda,Puentes Viejas,Puebla de la Sierra


---

### Ahora vamos a hacer scrapping para obtener la poblacion y la superficie por municipio

In [5]:
# Elemento que contiene la tabla
table = pagina.find('table', class_='wikitable sortable')

# Pasamos la tabla a dataframe
df_habitantes = pd.read_html(str(table), thousands='.', decimal= ',')[0]

# Tenemos que preparar y quitar errores que haya en el dataframe
df_habitantes['Población'] = df_habitantes['Población'].str.split().str.join('')

# Columna densidad de poblacion por municipio
df_habitantes['Densidad'] = round(df_habitantes['Población'].astype(np.number) / df_habitantes['Superficie'].astype(np.number), 2)

# Lo guardamos como csv
df_habitantes.to_csv('./csv/Habitantes.csv', encoding='latin1')
df_habitantes.head()

,Municipio,Superficie,Población,Densidad
0,Total comarca,1252.89,31930,25.49
1,Alameda del Valle,25.01,246,9.84
2,Berzosa del Lozoya,14.32,216,15.08
3,Braojos de la Sierra,24.93,218,8.74
4,Buitrago del Lozoya,26.50,1940,73.21


---

### *Mergeamos* toda la informacion obtenida

In [6]:
# Esta funcion nos permite encontrar a que subcomarca pertenece cada municipio
def subcomarca_municipio(municipio, dic):
    for key, valor in dic.items():
        if municipio in valor:
            return key

In [7]:
# Creamos el DataFrame 'pre'
df_pre = df_habitantes.copy()
df_pre.insert(0, 'Subcomarca', df_habitantes['Municipio'].apply(subcomarca_municipio, args= [subcomarcas]))


In [8]:
# Comprobamos si algo ha fallado
print(df_pre.isna().sum()) # Vemos que hay 2 null en Subcomarca
print(df_pre[df_pre.isnull().any(axis=1)]) # No se ha imputado bien la subcomarca de 'Horcajo de la Sierra-Aoslos', lo hacemos manualmente
row = df_pre.index[df_pre['Municipio'] == 'Horcajo de la Sierra-Aoslos'] # Sacamos el indice (No hace falta pero queda más profesional, supongo...)
df_pre.loc[row, 'Subcomarca'] = 'Valle Medio del Lozoya'

Subcomarca    2
Municipio     0
Superficie    0
Población     0
Densidad      0
dtype: int64
   Subcomarca                    Municipio  Superficie Población  Densidad
0        None                Total comarca     1252.89     31930     25.49
15       None  Horcajo de la Sierra-Aoslos       20.57       170      8.26


In [9]:
# Ponemos bonito el dataframe
df_pre.rename(columns = {'Superficie': 'Superficie (Km²)'}, inplace=True)
df_pre= df_pre.sort_values('Subcomarca').reset_index(drop=True)

# Lo guardamos
df_pre.to_csv('./csv/Pre.csv', encoding='latin1')

Ahora continuaremos en otro notebook para hacer scrapping de las geolocalizacion

---

Importamos el csv con los nuevos datos

In [10]:
df_coor = pd.read_csv('./csv/Geo.csv', encoding='latin1', index_col='Unnamed: 0')

Mergeamos ambos dataframes

In [13]:
df_final = pd.merge(df_pre, df_coor, how='inner', left_on='Municipio', right_on='Municipio')
df_final['C_Postal'] = df_final['C_Postal'].astype('Int16')
df_final

,Subcomarca,Municipio,Superficie (Km²),Población,Densidad,C_Postal,Latitud,Longitud
0,Sierra de La Cabrera,Valdemanco,17.58,1012,57.57,28729,40.872299,-3.662958
1,Sierra de La Cabrera,Bustarviejo,57.32,2701,47.12,28720,40.857568,-3.709243
2,Sierra de La Cabrera,Cabanillas de la Sierra,14.07,837,59.49,28721,40.822029,-3.627606
3,Sierra de La Cabrera,Navalafuente,11.75,1573,133.87,28729,40.823341,-3.674844
4,Sierra de La Cabrera,Lozoyuela-Navas-Sieteiglesias,51.28,1407,27.44,28752,40.927044,-3.618350
5,Sierra de La Cabrera,Venturada,9.79,2387,243.82,28729,40.799050,-3.622413
6,Sierra de La Cabrera,La Cabrera,22.40,2782,124.20,28751,40.865589,-3.615448
7,Sierra del Rincón,Puebla de la Sierra,57.70,75,1.30,28190,41.011705,-3.445102
8,Sierra del Rincón,Prádena del Rincón,22.48,149,6.63,28191,41.044075,-3.540475
9,Sierra del Rincón,Montejo de la Sierra,31.95,365,11.42,28190,41.059505,-3.529728


Descripcion

In [16]:
df_final.describe()

,Superficie (Km²),Densidad,C_Postal,Latitud,Longitud
count,42.000000,42.000000,42.0,42.000000,42.000000
mean,29.894286,31.023095,28556.642857,40.952384,-3.617718
std,23.815437,47.220450,262.162473,0.093523,0.101821
min,5.440000,1.300000,28180.0,40.767439,-3.879530
25%,18.070000,6.952500,28193.25,40.876552,-3.669070
50%,23.710000,11.545000,28729.5,40.953840,-3.609359
75%,33.592500,27.590000,28742.75,41.027627,-3.541851
max,150.280000,243.820000,28756.0,41.133003,-3.445102
